## 1 Generate data

### Generate the data in Gauss Distribution

In [7]:
import numpy as np
import matplotlib.pyplot as plt

SHOW_PLOT_IN_BLOCK: bool = False


def save_fig(filename: str, show: bool = SHOW_PLOT_IN_BLOCK) -> None:
    path = "../Report/img/Part1/" + filename + ".png"
    plt.savefig(path)
    if show:
        plt.show()


def generate_data(
    mean1: tuple[int, int],
    mean2: tuple[int, int],
    cov1: tuple[tuple[int, int], tuple[int, int]],
    cov2: tuple[tuple[int, int], tuple[int, int]],
    seed: int = 42,
) -> tuple[np.ndarray, np.ndarray]:
    if seed != 0:
        np.random.seed(seed)
    print(f"Mean1:{mean1}, Mean2:{mean2}")
    print(f"Cov1:{cov1}, Cov2: {cov2}")
    p_pos = np.random.multivariate_normal(mean1, cov1, 100)
    p_neg = np.random.multivariate_normal(mean2, cov2, 100)

    return p_pos, p_neg

### Shuffle the data function

In [8]:
def shuffle_data(
    points1: np.ndarray, points2: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    train_points_pos, test_points_pos = points1[:80], points1[80:]
    train_points_neg, test_points_neg = points2[:80], points2[80:]

    train_points = np.concatenate((train_points_pos, train_points_neg))
    train_labels = np.concatenate((np.ones(80), -1 * np.ones(80)))
    test_points = np.concatenate((test_points_pos, test_points_neg))
    test_labels = np.concatenate((np.ones(20), -1 * np.ones(20)))
    permutation = np.random.permutation(train_points.shape[0])
    shuffled_train = train_points[permutation, :]
    shuffled_label = train_labels[permutation]
    return shuffled_train, shuffled_label, test_points, test_labels

## 2 Train

### Add Plot Function

In [9]:
def plot_test(
    train_points: np.ndarray,
    train_labels: np.ndarray,
    test_points: np.ndarray,
    test_labels: np.ndarray,
    weight: np.ndarray,
    test_id: int,
) -> None:
    train_pos = train_points[train_labels == 1]
    train_neg = train_points[train_labels == -1]
    test_pos = test_points[test_labels == 1]
    test_neg = test_points[test_labels == -1]
    plt.scatter(
        train_pos[:, 0], train_pos[:, 1], c="blue", marker="o", label="positive train"
    )
    plt.scatter(
        train_neg[:, 0], train_neg[:, 1], c="red", marker="o", label="negative train"
    )
    plt.scatter(
        test_pos[:, 0], test_pos[:, 1], c="blue", marker="x", label="positive test"
    )
    plt.scatter(
        test_neg[:, 0], test_neg[:, 1], c="red", marker="x", label="negative test"
    )

    b, w1, w2 = weight
    x_min, x_max = test_points[:, 0].min(), test_points[:, 0].max()
    x_range = np.linspace(x_min, x_max, 100)
    y_range = (-w1 / w2) * x_range - (b / w2)
    plt.plot(
        x_range, y_range, color="black", label="Boundary", linewidth=2, linestyle="--"
    )
    plt.xlabel("point-x")
    plt.ylabel("point-y")
    plt.legend()
    plt.title(f"Test Case {test_id}")
    save_fig(f"test{test_id}_data", show=SHOW_PLOT_IN_BLOCK)
    plt.close()

### Add Show Curve Function

In [10]:
def show_curve(
    train_acc: list[float],
    train_loss: list[float],
    test_acc: list[float],
    test_loss: list[float],
    test_id: int,
) -> None:
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(train_acc, label="Train Accuracy")
    plt.plot(test_acc, label="Test Accuracy")
    plt.title(f"Test id {test_id} Training and Test Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy(%)")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(train_loss, label="Train Loss")
    plt.plot(test_loss, label="Test Loss")
    plt.title(f"Test id {test_id} Training and Test Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    save_fig(f"test{test_id}_curve", show=SHOW_PLOT_IN_BLOCK)
    plt.close()

### Add Test Function

In [11]:
from perceptron import Perceptron


def test_once(data: tuple[np.ndarray, np.ndarray], test_id: int) -> None:
    points1, points2 = data

    shuffled_train, shuffled_label, test_points, test_labels = shuffle_data(
        points1, points2
    )

    p: Perceptron = Perceptron(2)
    train_acc, train_loss, test_acc, test_loss = p.train(
        shuffled_train, shuffled_label, test_points, test_labels
    )

    plot_test(
        shuffled_train, shuffled_label, test_points, test_labels, p.weights, test_id
    )
    show_curve(train_acc, train_loss, test_acc, test_loss, test_id)


def different_test() -> None:
    print("Test id 1")
    mean1, mean2 = (3, 3), (15, 15)
    cov1, cov2 = ((2, 0), (0, 2)), ((2, 0), (0, 2))
    test_once(generate_data(mean1, mean2, cov1, cov2), 1)

    print("Test id 2")
    mean1, mean2 = (3, 3), (6, 6)
    cov1, cov2 = ((10, 0), (0, 10)), ((10, 0), (0, 10))
    test_once(generate_data(mean1, mean2, cov1, cov2), 2)

    print("Test id 3")
    mean1, mean2 = (3, 3), (4, 4)
    cov1, cov2 = ((2, 0), (0, 2)), ((2, 0), (0, 2))
    test_once(generate_data(mean1, mean2, cov1, cov2), 3)


different_test()

Test id 1
Mean1:(3, 3), Mean2:(15, 15)
Cov1:((2, 0), (0, 2)), Cov2: ((2, 0), (0, 2))
Final test loss:0.000000, acc: 100.00%
Test id 2
Mean1:(3, 3), Mean2:(6, 6)
Cov1:((10, 0), (0, 10)), Cov2: ((10, 0), (0, 10))
Final test loss:0.686886, acc: 80.00%
Test id 3
Mean1:(3, 3), Mean2:(4, 4)
Cov1:((2, 0), (0, 2)), Cov2: ((2, 0), (0, 2))
Final test loss:6.698976, acc: 50.00%


### Static Show Test Example

![test1_data](../Report/img/Part1/test1_data.png)

![test1_curve](../Report/img/Part1/test1_curve.png)

![test2_data](../Report/img/Part1/test2_data.png)

![test2_curve](../Report/img/Part1/test2_curve.png)

![test3_data](../Report/img/Part1/test3_data.png)

![test3_curve](../Report/img/Part1/test3_curve.png)